In [1]:
import os

path_parent = os.path.dirname(os.getcwd())
os.chdir(path_parent)

In [11]:
import pandas as pd
import numpy as np
from app.utils.visualization import plot_data
from app.utils.fetch_data import fred_fred, investing_api, alpha_vantage_api, alpha_vantage_api_financial_statements
from app.utils.fetch_data_fmp import FMP

In [ ]:
import investpy
from datetime import datetime, date
from scipy.stats import pearsonr
from sklearn import preprocessing
from statsmodels.regression.linear_model import OLS
from sklearn.ensemble import RandomForestRegressor

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
start_date = '2003-01-01'
today = date.today().strftime("%Y-%m-%d")

## Gather data - target

## Gather data - monthly

In [5]:
df_dict = dict()

df_dict['ir10y'] = fred_fred('REAINTRATREARAT10Y', start_date, today) # percent
df_dict['ir1y'] = fred_fred('REAINTRATREARAT1YE', start_date, today) # percent

df_dict['cpi'] = fred_fred('CPALTT01USM657N', start_date, today) # growth rate previous period
df_dict['inflation10y'] = fred_fred('T10YIEM', start_date, today) # percent
df_dict['inflation5y'] = fred_fred('T5YIEM', start_date, today) # percent

df_dict['inflation_exp'] = fred_fred('MICH', start_date, today) # percent

df_dict['unemp_rate'] = fred_fred('UNRATENSA', start_date, today) # percent

df_dict['fed_ir'] = fred_fred('FEDFUNDS', start_date, today)

# df_dict['gdp_us'] = fred_fred('USALORSGPNOSTSAM', start_date, today)

2022-10-01 15:09:00,166 [app.utils.fetch_data] [INFO] Fetching data from fred: REAINTRATREARAT10Y, from 2003-01-01 to 2022-10-01.
2022-10-01 15:09:02,344 [app.utils.fetch_data] [INFO] Fetching data from fred: REAINTRATREARAT1YE, from 2003-01-01 to 2022-10-01.
2022-10-01 15:09:03,769 [app.utils.fetch_data] [INFO] Fetching data from fred: CPALTT01USM657N, from 2003-01-01 to 2022-10-01.
2022-10-01 15:09:05,295 [app.utils.fetch_data] [INFO] Fetching data from fred: T10YIEM, from 2003-01-01 to 2022-10-01.
2022-10-01 15:09:06,659 [app.utils.fetch_data] [INFO] Fetching data from fred: T5YIEM, from 2003-01-01 to 2022-10-01.
2022-10-01 15:09:08,049 [app.utils.fetch_data] [INFO] Fetching data from fred: MICH, from 2003-01-01 to 2022-10-01.
2022-10-01 15:09:09,502 [app.utils.fetch_data] [INFO] Fetching data from fred: UNRATENSA, from 2003-01-01 to 2022-10-01.
2022-10-01 15:09:10,935 [app.utils.fetch_data] [INFO] Fetching data from fred: FEDFUNDS, from 2003-01-01 to 2022-10-01.


## Gather data - daily

In [8]:
df_dict['inflation5y']

,Date,v,symbol,type,interval,unit,p_key
0,2003-01-01,1.40,T5YIEM,index,monthly,percent,2003_01_01_T5YIEM
1,2003-02-01,1.66,T5YIEM,index,monthly,percent,2003_02_01_T5YIEM
2,2003-03-01,1.69,T5YIEM,index,monthly,percent,2003_03_01_T5YIEM
3,2003-04-01,1.57,T5YIEM,index,monthly,percent,2003_04_01_T5YIEM
4,2003-05-01,1.34,T5YIEM,index,monthly,percent,2003_05_01_T5YIEM
...,...,...,...,...,...,...,...
231,2022-04-01,3.32,T5YIEM,index,monthly,percent,2022_04_01_T5YIEM
232,2022-05-01,3.03,T5YIEM,index,monthly,percent,2022_05_01_T5YIEM
233,2022-06-01,2.89,T5YIEM,index,monthly,percent,2022_06_01_T5YIEM
234,2022-07-01,2.58,T5YIEM,index,monthly,percent,2022_07_01_T5YIEM


In [6]:
df_dict['cpi']

,Date,v,symbol,type,interval,unit,p_key
0,2003-01-01,0.442233,CPALTT01USM657N,index,monthly,growth rate previous period,2003_01_01_CPALTT01USM657N
1,2003-02-01,0.770501,CPALTT01USM657N,index,monthly,growth rate previous period,2003_02_01_CPALTT01USM657N
2,2003-03-01,0.600765,CPALTT01USM657N,index,monthly,growth rate previous period,2003_03_01_CPALTT01USM657N
3,2003-04-01,-0.217155,CPALTT01USM657N,index,monthly,growth rate previous period,2003_04_01_CPALTT01USM657N
4,2003-05-01,-0.163221,CPALTT01USM657N,index,monthly,growth rate previous period,2003_05_01_CPALTT01USM657N
...,...,...,...,...,...,...,...
230,2022-03-01,1.335138,CPALTT01USM657N,index,monthly,growth rate previous period,2022_03_01_CPALTT01USM657N
231,2022-04-01,0.558253,CPALTT01USM657N,index,monthly,growth rate previous period,2022_04_01_CPALTT01USM657N
232,2022-05-01,1.102352,CPALTT01USM657N,index,monthly,growth rate previous period,2022_05_01_CPALTT01USM657N
233,2022-06-01,1.373608,CPALTT01USM657N,index,monthly,growth rate previous period,2022_06_01_CPALTT01USM657N
